<a href="https://colab.research.google.com/github/dolts/eav_smspl/blob/main/Eventos_Adversos_a_Vacina_SMS_PMPL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Renomeando Arquivo .csv

In [ ]:
! mv *.csv eav.csv

mv: 'eav.csv' and 'eav.csv' are the same file


# Pegando dados do arquivo de números no Google Drive

#### Precisa estar com o Drive montado

In [ ]:
# Iniciando leitura do arquivo e dividindo em uma lista
with open('drive/MyDrive/numeros_eav.txt') as numbers_file:
    number_to_exclude = numbers_file.read().split('\n')

# Iniciando análise com o pandas

In [ ]:
import pandas as pd

data = pd.read_csv('eav.csv', sep=';')

In [ ]:
index_to_drop = data.query('`Número da Notificação` in @number_to_exclude').index
data.drop(index_to_drop, inplace=True)

In [ ]:
fields = [
          'Número da Notificação',
          'Nome Completo',
          'Notificante CNES',
          'Notificante Nome Completo'
]

In [ ]:
data_filtered = data.query('`Município de Residência` == "Pedro Leopoldo"')[fields]

In [ ]:
# Definindo nome do arquivo e salvando
from datetime import datetime
from dateutil import tz

time_now = datetime.now()
time_now = time_now.astimezone(tz.gettz('America/Sao_Paulo'))

final_file_name = f'eav_{time_now.strftime("%d_%m_%Y")}.xlsx'

with pd.ExcelWriter(final_file_name) as writer:
    data_filtered.to_excel(writer, sheet_name='EAV', index=False)

# Fazendo download da planilha via código

In [ ]:
from google.colab import files

files.download(final_file_name)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Salvando os números gerados automaticamente no arquivo importado

In [ ]:
with open('drive/MyDrive/numeros_eav.txt', 'a') as numbers_file:
    for number in data_filtered['Número da Notificação']:
        numbers_file.write(f'{str(number)}\n')


# Pegando credenciais para envio de e-mail

In [ ]:
import json

with open('drive/MyDrive/credentials.json', 'r') as file:
    credentials = json.loads(file.read())

# Definindo mensagem e "montando" e-mail para envio

In [ ]:
message = f'''
Prezada,

Segue anexo relatório feito no dia {time_now.strftime("%d/%m/%Y")} às {time_now.strftime("%H:%M:%S")}.
Este é um envio automático, caso hava alguma discrepância favor entrar em contato.

Atenciosamente,

Douglas Andrade de Oliveira
Coordenação de Atenção Primária
Secretaria Municipal de Saúde
Pedro Leopoldo - MG
(31) 3660-5132
'''.encode('utf-8')

In [ ]:
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders

host = 'smtp.gmail.com'
port = 465

server = smtplib.SMTP_SSL(host, port)
server.login(credentials['email'], credentials['password'])

email_msg = MIMEMultipart()
email_msg['From'] = credentials['email']
email_msg['To'] = credentials['receiver']
email_msg['Subject'] = f'Relatório E-SUS Notifica EAV {time_now.strftime("%d/%m/%Y")}'
email_msg.attach(MIMEText(message.decode(encoding='utf-8'), 'plain'))

attachment = open(final_file_name, 'rb')

att = MIMEBase('application', 'octet-stream')
att.set_payload(attachment.read())
encoders.encode_base64(att)
att.add_header('Content-Disposition', f'attachment; filename= {final_file_name}')

attachment.close()

email_msg.attach(att)

server.sendmail(email_msg['From'], email_msg['To'], email_msg.as_string())

server.quit()

(221, b'2.0.0 closing connection k14sm2064557uae.12 - gsmtp')